# Generate EMA using dedicated service - Python

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *exponential moving average - EMA* from a dedicated data service.

### Inputs/outputs
Exponential moving averages' requests require instrument's identifier, date time intervals and number of points as per inputs.

EMA reduces the lag by applying greater weight to recent data points.The weighting applied to the most recent price depends on the number of points in the moving average; The smaller the point count is, the more weight is applied to the most recent price.

N-points EMA data point is computed using the previous' *EMA* value and the wighting multiplier as follows:

$$ EMA = (Price-EMAp)*k+EMAp $$

Where:
* *Price* is the trade price
* *EMAp* is the previous points' *EMA* value, *SMA* is used as the EMAp in the first calculation
* *N* is the count of points
* *$ k = \frac{2}{N+1} $* is the *Weighting Multiplier*

<div class="alert alert-block alert-info">
    <b>Note:</b> <i>EMA</i> adapts quickly to price changes than the <i>SMA</i>, when a price reverses direction, the <i>EMA</i> will reverse direction quicker than the <i>SMA</i> dur to the weighting 
</div>

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve EMA indicator from the hosted service. The queried endpoint in this script are:
* *TickEmaService*: to directly retrieve EMA data from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display packages

***

# Run EMA sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly matplotlib systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_ema_pb2 as tick_ema
import systemathics.apis.services.tick_analytics.v1.tick_ema_pb2_grpc as tick_ema_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request
To request *Tick EMA* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Exponential moving average request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 EMA parameters

In [ ]:
field = tick_ema.EMA_PRICE_TRADE; # set the EMA calculation field
long_ema = 200; # set the long EMA length
short_ema = 50; # set the short EMA length
sampling = 1 * 60

#### 3.3 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2020, month = 2, day = 18), 
    end_date = date.Date(year = 2020, month = 2, day = 18)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 8, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 0, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

#### 3.4 Long EMA request creation
The following code snippets creates *gRPC client*, process *long EMA* request and returns the request reply:

In [ ]:
# generate the tick long EMA request
long_request = tick_ema.TickEmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints,
    field = field,
    length = long_ema,
    sampling = duration.Duration(seconds = sampling)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel(os.environ("GRPC_APIS"), credentials) as channel:
    
    # instantiate the tick EMA service
    service = tick_ema_service.TickEmaServiceStub(channel)
    
    long_averages = []
    # process the tick long EMA request
    for l in service.TickEma(request = long_request, metadata = [('authorization', token)]):
        if l.average.value > 0:  # we have an average
            long_averages.append(l.average.value)
        else: 
            long_averages.append(None)

#### 3.4 Short EMA request creation
The following code snippets creates *gRPC client*, process *short EMA* request and returns the request reply:

In [ ]:
# generate the tick short EMA request
short_request = tick_ema.TickEmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints,
    field = field,
    length = short_ema,
    sampling = duration.Duration(seconds = sampling)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel(os.environ("GRPC_APIS"), credentials) as channel:
    
    # instantiate the tick EMA service
    service = tick_ema_service.TickEmaServiceStub(channel)
    
    time_stamps, short_averages, prices = [], [], []
    # process the tick short SMA request
    for s in service.TickEma(request = short_request, metadata = [('authorization', token)]):
        time_stamps.append(s.time_stamp)
        prices.append(s.value)
        if s.average.value > 0:  # we have an average
            short_averages.append(s.average.value)
        else: 
            short_averages.append(None)

### Step 4: Visualize data

#### 4.1 Retrieve EMAs data

In [ ]:
# create the SMAs dataframe
dates=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps]
d = {'Date': dates, 'Long Ema': long_averages, 'Short Ema': short_averages, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot tick EMAs and prices with matplotlib

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Long Ema', data=df, marker='', color='orange', linewidth=2, label="Long EMA")
ax.plot('Date', 'Short Ema', data=df, marker='', color='green', linewidth=2, label="Short EMA")
ax.plot('Date', 'Price', data=df, marker='', color='blue', linewidth=2, label="Price")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Long & short tick EMA for {0}-{1}'.format(ticker,exchange))
plt.show()

#### 4.3 Plot tick EMAs and prices with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Short Ema'], name='Short Ema', marker_color ='green'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Long Ema'], name='Long Ema', marker_color ='red'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price', marker_color ='blue'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick EMAs and prices for {0}-{1}'.format(ticker,exchange))
fig.show()